### Classification on the Penguin Dataset

#### Libraries

In [7]:
import pandas as pd 

import h2o

#### Read data

In [8]:
penguins = pd.read_csv("data/penguins.txt")

In [9]:
penguins.head(3)

,studyName,Sample Number,Species,Region,Island,Stage,Individual ID,Clutch Completion,Date Egg,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Comments
0,PAL0708,1,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A1,Yes,11/11/07,39.1,18.7,181.0,3750.0,MALE,NaN,NaN,Not enough blood for isotopes.
1,PAL0708,2,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A2,Yes,11/11/07,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454,NaN
2,PAL0708,3,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A1,Yes,11/16/07,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302,NaN


#### Preprocess data

We will only use this four columns for analysis : 
- Culmen length (mm)
- Culmen Depth (mm)
- Flipper Length (mm)
- Body Mass (g)

In [10]:
attributes = ["Culmen Length (mm)","Culmen Depth (mm)","Flipper Length (mm)","Body Mass (g)"]
target = ["Species"]
penguins = penguins[attributes+target]
penguins.dropna(inplace=True)

#### Initialize H2o Cluster

In [11]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,39 mins 20 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.4
H2O_cluster_version_age:,5 days
H2O_cluster_name:,H2O_from_python_massonga_h1rso7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.423 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


#### Train-Test Split

In [12]:
penguins = h2o.H2OFrame(penguins)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [13]:
train, test, valid = penguins.split_frame(ratios = [.7, .15], seed = 1234)
x = attributes
y = "Species"

#### Classification with auto ML

In [14]:
aml = h2o.automl.H2OAutoML(max_models = 10, seed = 1,max_runtime_secs=180)

In [15]:
aml.train(x=x, y=y, training_frame=train, validation_frame=valid)

AutoML progress: |█
14:32:14.365: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
14:32:14.378: AutoML: XGBoost is not available; skipping it.
14:32:17.41: GBM_1_AutoML_2_20230111_143214 [GBM def_5] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_1_AutoML_2_20230111_143214.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 199.0.
ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 199.0.
ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 199.0.
ERRR

Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_1_AutoML_2_20230111_143214


GLM Model: summary
    family       link         regularization               lambda_search                                                                    number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  -----------  -----------  ---------------------------  -------------------------------------------------------------------------------  ----------------------------  -----------------------------  ----------------------  -----------------------------------------------
    multinomial  multinomial  Ridge ( lambda = 1.102E-4 )  nlambda = 30, lambda.max = 42.501, lambda.min = 1.102E-4, lambda.1se = 0.001193  15                            12                             96                      AutoML_2_20230111_143214_training_py_2_sid_af0b

ModelMetricsMultinomialGLM: glm
** Reported on train data. **

MSE: 0.0028033737507429848
RMSE: 0.05294689557229002
LogLoss: 0.012141395097790667
Null degrees of freedom: 248
Residual degrees of freedom: 234
Null deviance: 520.974134731867
Residual deviance: 6.046414758699752
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Adelie Penguin (Pygoscelis adeliae)    Chinstrap penguin (Pygoscelis antarctica)    Gentoo penguin (Pygoscelis papua)    Error    Rate
-------------------------------------  -------------------------------------------  -----------------------------------  -------  -------
110                                    0                                            0                                    0        0 / 110
0                                      48                                           0                                    0        0 / 48
0                                      0                                            91                                   0        0 / 91
110                                    48                                           91                                   0        0 / 249

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    1
2    1
3    1

ModelMetricsMultinomialGLM: glm
** Reported on validation data. **

MSE: 0.00620529980556978
RMSE: 0.07877372535033353
LogLoss: 0.01783939199754221
Null degrees of freedom: 53
Residual degrees of freedom: 39
Null deviance: 113.30353005398744
Residual deviance: 1.9266543357345585
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Adelie Penguin (Pygoscelis adeliae)    Chinstrap penguin (Pygoscelis antarctica)    Gentoo penguin (Pygoscelis papua)    Error      Rate
-------------------------------------  -------------------------------------------  -----------------------------------  ---------  ------
25                                     0                                            0                                    0          0 / 25
1                                      10                                           0                                    0.0909091  1 / 11
0                                      0                                            18                                   0          0 / 18
26                                     10                                           18             

#### Compare all the models

In [16]:
aml.leaderboard.as_data_frame().sort_values(by='rmse')

,model_id,mean_per_class_error,logloss,rmse,mse
0,GLM_1_AutoML_2_20230111_143214,0.016919,0.028084,0.096912,0.009392
1,StackedEnsemble_BestOfFamily_1_AutoML_2_202301...,0.016919,0.046786,0.106619,0.011368
6,DeepLearning_1_AutoML_2_20230111_143214,0.026894,0.052384,0.111788,0.012497
2,StackedEnsemble_AllModels_1_AutoML_2_20230111_...,0.023864,0.054036,0.115320,0.013299
10,DeepLearning_grid_1_AutoML_2_20230111_143214_m...,0.034722,0.070598,0.130618,0.017061
4,GBM_2_AutoML_2_20230111_143214,0.026894,0.060541,0.131091,0.017185
3,XRT_1_AutoML_2_20230111_143214,0.026894,0.072608,0.132453,0.017544
5,DRF_1_AutoML_2_20230111_143214,0.026894,0.076056,0.136338,0.018588
7,GBM_5_AutoML_2_20230111_143214,0.029924,0.066401,0.137028,0.018777
9,GBM_3_AutoML_2_20230111_143214,0.029924,0.065016,0.137063,0.018786


#### Get the best model tested

In [19]:
model = h2o.get_model("GLM_1_AutoML_2_20230111_143214")
model.model_performance(test)

ModelMetricsMultinomialGLM: glm
** Reported on test data. **

MSE: 0.001117375420704516
RMSE: 0.0334271659089507
LogLoss: 0.008946366609923503
Null degrees of freedom: 38
Residual degrees of freedom: 24
Null deviance: 83.96026984007607
Residual deviance: 0.6978165955740333
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Adelie Penguin (Pygoscelis adeliae)    Chinstrap penguin (Pygoscelis antarctica)    Gentoo penguin (Pygoscelis papua)    Error    Rate
-------------------------------------  -------------------------------------------  -----------------------------------  -------  ------
16                                     0                                            0                                    0        0 / 16
0                                      9                                            0                                    0        0 / 9
0                                      0                                            14                                   0        0 / 14
16                                     9                                            14                                   0        0 / 39

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    1
2    1
3    1